Nama Anggota Kelompok:

Mochammad Rizki Aji Santoso (200411100086)
Mohammad Unis Iswahyudi     (200411100185)

In [1]:
import pandas as pd
from numpy import array

df = pd.read_csv('/home/ubuntu/workspace/transactions.csv')
df

,date,transactions
0,2013-01-01,770
1,2013-01-02,2111
2,2013-01-02,2358
3,2013-01-02,3487
4,2013-01-02,1922
...,...,...
83483,2017-08-15,2804
83484,2017-08-15,1573
83485,2017-08-15,2255
83486,2017-08-15,932


In [2]:
data = df['transactions']
data

0         770
1        2111
2        2358
3        3487
4        1922
         ... 
83483    2804
83484    1573
83485    2255
83486     932
83487     802
Name: transactions, Length: 83488, dtype: int64

In [3]:
n = len(data)
sizeTrain = (round(n*0.8))
data_train = pd.DataFrame(data[:sizeTrain])
data_test = pd.DataFrame(data[sizeTrain:])
data_train

,transactions
0,770
1,2111
2,2358
3,3487
4,1922
...,...
66785,2037
66786,469
66787,1420
66788,1193


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(data_train)


# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = sc.fit_transform(data_train)

# Mengaplikasikan MinMaxScaler pada data pengujian
test_scaled = scaler.transform(data_test)

# reshaped_data = data.reshape(-1, 1)
train = pd.DataFrame(train_scaled, columns = ['data'])
train = train['data']

test = pd.DataFrame(test_scaled, columns = ['data'])
test = test['data']
test



0        0.074096
1        0.161599
2        0.066076
3        0.116711
4        0.287048
           ...   
16693    0.335049
16694    0.187695
16695    0.269332
16696    0.110965
16697    0.095403
Name: data, Length: 16698, dtype: float64

In [5]:
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)

In [6]:
df_X, df_Y = split_sequence(train, 2)
# for i in range(len(df_X)):
#   print(df_X[i], df_Y[i])
x = pd.DataFrame(df_X, columns = ['xt-2','xt-1'])
y = pd.DataFrame(df_Y, columns = ['xt'])
dataset_train = pd.concat([x, y], axis=1)
dataset_train
dataset_train.to_excel('data-train.xlsx', index=False)


In [7]:
X_train = dataset_train.iloc[:, :2].values
Y_train = dataset_train.iloc[:, -1].values
X_train

array([[0.0915729 , 0.2520948 ],
       [0.2520948 , 0.28166148],
       [0.28166148, 0.41680632],
       ...,
       [0.24323677, 0.05554226],
       [0.05554226, 0.16937994],
       [0.16937994, 0.14220733]])

In [8]:
test_x, test_y = split_sequence(test, 2)
# for i in range(len(test_x)):
#   print(test_x[i], test_y[i])
x = pd.DataFrame(test_x, columns = ['xt-2','xt-1'])
y = pd.DataFrame(test_y, columns = ['xt'])
dataset_test = pd.concat([x, y], axis=1)
dataset_test
dataset_test.to_excel('data-test.xlsx', index=False)


In [9]:
X_test = dataset_test.iloc[:, :2].values
Y_test = dataset_test.iloc[:, -1].values
print(len(X_test))


16696


In [10]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(X_train,Y_train)

KNeighborsRegressor(n_neighbors=3)

In [11]:
y_pred=neigh.predict(X_test)
reshaped_data = y_pred.reshape(-1, 1)
original_data = scaler.inverse_transform(reshaped_data)
reshaped_datates = Y_test.reshape(-1, 1)
actual_test = scaler.inverse_transform(reshaped_datates)
# print(original_data)
# print(actual_test)
akhir1 = pd.DataFrame(original_data)
akhir1.to_excel('prediksi.xlsx', index=False)
akhir = pd.DataFrame(actual_test)
akhir.to_excel('aktual.xlsx', index=False)

In [12]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(original_data, actual_test)
print(mean_absolute_percentage_error(original_data, actual_test))

0.3247941135893394
